In [ ]:
import arcpy
from arcpy.sa import *
import os
import time
arcpy.env.overwriteOutput = True

In [ ]:
# load input features and make layer
version='v11_240805'
stastic_v=f'{version}_v1'
output_dir = os.path.join(r'J:\lakemapping\postprocess',version)
output_2=os.path.join(r'J:\lakemapping\postprocess','v10_240710')

auxiliary_dataset_gdb=r'D:\postprocess\v240220\auxiliary_dataset.gdb'
after_mask_gdb=os.path.join(output_dir,'3_polygon_after_rivermask.gdb')
stastics_gdb=os.path.join(output_dir,'0_summary_stastics.gdb')
mwBL_gdb=os.path.join(output_dir,'4_polygon_afm_mergeWith_BigLake.gdb')
mwBG_gdb=os.path.join(output_dir,'5_polygon_afm_mwBL_BigGLAKES.gdb')
excel_dir=os.path.join(output_dir,'stastics_excel')

if not os.path.exists(mwBL_gdb):
    arcpy.CreateFileGDB_management(output_dir,'4_polygon_afm_mergeWith_BigLake.gdb')
if not os.path.exists(mwBG_gdb):
    arcpy.CreateFileGDB_management(output_dir,'5_polygon_afm_mwBL_BigGLAKES.gdb')
if not os.path.exists(stastics_gdb):
    arcpy.CreateFileGDB_management(output_dir,'0_summary_stastics.gdb')
if not os.path.exists(after_mask_gdb):
    arcpy.CreateFileGDB_management(output_dir,'3_polygon_after_rivermask.gdb')

In [ ]:
lakes_iwBR=r'J:\lakemapping\postprocess\v10_240710\2_polygon_iw_River.gdb\total_iwBR_arm_SJ'
total_lakes_arm=r'J:\lakemapping\postprocess\v10_240710\2_polygon_iw_River.gdb\total_lakes_arm'
lakes_iwBR_afterMask=os.path.join(after_mask_gdb,'lakes_iwBR_afterMask')
lakes_iwBR_lyr='lakes_iwBR'
total_lakes_arm_lyr='lakes_arm'
lakes_iwBR_afm_lyr='lakes_iwBR_afm'

In [ ]:
a_iwBL_merge_with_BL_raw=os.path.join(after_mask_gdb,'a_total_lakes_afm_iwBL_merge_with_BigLakes_raw')
a_iwBL_merge_with_BL=os.path.join(after_mask_gdb,'a_total_lakes_afm_iwBL_merge_with_BigLakes')
a_iwBL_merge_with_BL_raw_lyr='a_lakes_afm_iwBL_mw_BL_raw'
a_iwBL_merge_with_BL_lyr='a_lakes_afm_iwBL_mw_BL'

b_iwBG=os.path.join(mwBL_gdb,'b_total_lakes_afm_mwBL_iwBG')
b_iwBG_point=os.path.join(mwBL_gdb,'b_total_lakes_afm_mwBL_iwBG_point')
b_iwBG_lyr='b_lakes_afm_mwBL_iwBG'
b_iwBG_point_lyr='b_lakes_afm_mwBL_iwBG_point'

b_iwBG_merge_with_BG_raw=os.path.join(mwBL_gdb,'b_total_lakes_afm_mwBL_iwBG_merge_with_BG_raw')
b_iwBG_merge_with_BG=os.path.join(mwBL_gdb,'b_total_lakes_afm_mwBL_iwBG_merge_with_BG')
b_iwBG_merge_with_BG_raw_lyr='b_lakes_afm_mwBL_iwBG_mw_BG_raw'
b_iwBG_merge_with_BG_lyr='b_lakes_afm_mwBL_iwBG_mw_BG'

b_iwBG_merge_with_BG_SJ=os.path.join(mwBL_gdb,'b_iwBG_merge_with_BG_SJ')
b_iwBG_merge_with_BG_SJ_SJ=os.path.join(mwBL_gdb,'b_iwBG_merge_with_BG_SJ_SJ')
b_iwBG_merge_with_BG_SJ_dissolve=os.path.join(mwBL_gdb,'b_iwBG_merge_with_BG_SJ_dissolve')
b_iwBG_merge_with_BG_SJ_GLAKES_SJ=os.path.join(mwBL_gdb,'b_iwBG_merge_with_BG_SJ_GLAKES_SJ')
b_iwBG_merge_with_BG_final=os.path.join(mwBL_gdb,'b_iwBG_merge_with_BG_final')
b_iwBG_merge_with_BG_SJ_lyr='b_lakes_afm_mwBL_iwBG_mw_BG_SJ'
b_iwBG_merge_with_BG_SJ_dissolve_lyr='b_iwBG_merge_with_BG_SJ_dissolve'
b_iwBG_merge_with_BG_SJ_GLAKES_SJ_lyr='b_iwBG_merge_with_BG_SJ_GLAKES_SJ'
b_iwBG_merge_with_BG_final_lyr='b_iwBG_merge_with_BG_final'

In [ ]:
seven_continents=['Asia','Africa','Europe','Oceania_Anta','North_America','South_America']

In [ ]:
def selectByLocation(inputFeature,overlapType,selectionFeature,selectionType='NEW_SELECTION'):
    arcpy.SelectLayerByLocation_management(inputFeature,overlapType,selectionFeature,selection_type=selectionType)
    
def selectByAttribute(inputFeature,selectionType='NEW_SELECTION',code=''):
    arcpy.management.SelectLayerByAttribute(inputFeature,selectionType,code)
    
def calculateField(inputFeature,field,code):
    arcpy.CalculateField_management(inputFeature, field,code)

def cal_continent(lyr):
    arcpy.AddField_management(lyr, 'Continent', "Text")
    for i in range(0,6):
        continent=seven_continents[i]
        selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
        selectByAttribute(lyr,'NEW_SELECTION',"Continent IS NULL")
        selectByLocation(lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
        calculateField(lyr,'Continent',f"'{continent}'")

In [ ]:
big_lakes=os.path.join(after_mask_gdb,f'total_big_lakes')
big_lakes_lyr=f'big_lakes'
GLAKES_gte1_Rser=os.path.join(mwBL_gdb,'GLAKES_gte1_Rser')
GLAKES_gte1_Rser_singlepart=os.path.join(mwBL_gdb,'GLAKES_gte1_Rser_singlepart')
GLAKES_gte1_Rser_point=os.path.join(mwBL_gdb,'GLAKES_gte1_Rser_point')
GLAKES_gte1_Rser_lyr='GLAKES_gte1_Rser'
GLAKES_gte1_Rser_point_lyr='GLAKES_gte1_Rser_point'

In [ ]:
GLAKES_Res=os.path.join(auxiliary_dataset_gdb,'GLAKES_Res')
arcpy.MakeFeatureLayer_management(GLAKES_Res,'GLAKES_Res')

# GLAKES_natural_lake=os.path.join(auxiliary_dataset_gdb,'GLAKES_gte1_natural_lake')
# arcpy.MakeFeatureLayer_management(GLAKES_natural_lake,'GLAKES_natural_lake')

In [ ]:
continent=r'D:\lakemapping\0_auxiliary_data\World_Continents\World_Continents.shp'
continent_lyr='continent'
arcpy.MakeFeatureLayer_management(continent,continent_lyr)

## 1， 对GLAKES进行分类，合并big_lakes，获得mask后结果

### 1.1 对GLAKES进行分类

In [ ]:
GLAKES=r'D:\lakemapping\0_auxiliary_data\GLAKES\GLAKES.gdb\GLAKES'
arcpy.MakeFeatureLayer_management(GLAKES,'GLAKES')

In [ ]:
selectByAttribute('GLAKES','NEW_SELECTION','"Area_bound" >= 1 OR "Rser_flag"=1')
arcpy.conversion.ExportFeatures('GLAKES',GLAKES_gte1_Rser)

In [ ]:
arcpy.MakeFeatureLayer_management(GLAKES_gte1_Rser,GLAKES_gte1_Rser_lyr)

In [ ]:
# arcpy.management.FeatureToPoint(GLAKES_gte1_Rser_lyr,GLAKES_gte1_Rser_point,'INSIDE')
arcpy.MakeFeatureLayer_management(GLAKES_gte1_Rser_point,GLAKES_gte1_Rser_point_lyr)

### 1.2 合并big_lakes

In [ ]:
big_lake_list=[]
for region in region_list:
    polygon_big_lake=os.path.join(after_mask_gdb,'big_lakes_'+region)
    arcpy.MakeFeatureLayer_management(polygon_big_lake,'big_lakes_'+region)
    big_lake_list.append('big_lakes_'+region)
# arcpy.Merge_management(big_lake_list,big_lakes)

In [ ]:
arcpy.MakeFeatureLayer_management(big_lakes,big_lakes_lyr)

### 1.3 获得mask后结果

In [ ]:
arcpy.MakeFeatureLayer_management(lakes_iwBR,lakes_iwBR_lyr)
arcpy.MakeFeatureLayer_management(total_lakes_arm,total_lakes_arm_lyr)

In [ ]:
selectByAttribute(lakes_iwBR_lyr,'NEW_SELECTION','"operate" = 0.5 OR operate IS NULL')
selectByLocation(total_lakes_arm_lyr, 'WITHIN', lakes_iwBR_lyr,'NEW_SELECTION')
selectByAttribute(total_lakes_arm_lyr,'SUBSET_SELECTION','"operate" IS NULL')
selectByAttribute(total_lakes_arm_lyr,'ADD_TO_SELECTION','"operate" = 1')
selectByAttribute(lakes_iwBR_lyr,'NEW_SELECTION','"operate">= 1')
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(lakes_iwBR_lyr)#未mask
fieldmappings_all.addTable(total_lakes_arm_lyr)
field_list = ['area','iwR','iwG','flag','Continent','operate']
for field_i in field_list:
        field_idx = fieldmappings_all.findFieldMapIndex(field_i)
        fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
arcpy.Merge_management([lakes_iwBR_lyr,total_lakes_arm_lyr],lakes_iwBR_afterMask,fieldmappings)
arcpy.MakeFeatureLayer_management(lakes_iwBR_afterMask,lakes_iwBR_afm_lyr)
selectByAttribute(lakes_iwBR_afm_lyr,'NEW_SELECTION','area IS NULL')
arcpy.CalculateField_management(lakes_iwBR_afm_lyr, "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")

In [ ]:
arcpy.AddField_management(lakes_iwBR_afm_lyr, 'iwBL', "Short")
selectByLocation(lakes_iwBR_afm_lyr,'INTERSECT',big_lakes_lyr)
calculateField(lakes_iwBR_afm_lyr,'iwBL',1)
selectByAttribute(lakes_iwBR_afm_lyr,'NEW_SELECTION','iwBL IS NULL')
calculateField(lakes_iwBR_afm_lyr,'iwBL',0)

In [ ]:
arcpy.MakeFeatureLayer_management(lakes_iwBR_afterMask,lakes_iwBR_afm_lyr)

In [ ]:
for i in range(0,6):
    continent=seven_continents[i]
    print(continent)
    lakes_niwBR=os.path.join(r'J:\lakemapping\postprocess\v10_240710\2_polygon_iw_River.gdb','lakes_'+continent+'_niwBR')
    lakes_niwBR_lyr='lakes_'+continent+'_niwBR'
    arcpy.MakeFeatureLayer_management(lakes_niwBR,lakes_niwBR_lyr)
    arcpy.AddField_management(lakes_niwBR_lyr, 'iwBL', "Short")
    selectByLocation(lakes_niwBR_lyr,'INTERSECT',big_lakes_lyr)
    calculateField(lakes_niwBR_lyr,'iwBL',1)
    selectByAttribute(lakes_niwBR_lyr,'NEW_SELECTION','iwBL IS NULL')
    calculateField(lakes_niwBR_lyr,'iwBL',0)

In [ ]:
input_table_afm_list=[]
for i in range(0,6):
    continent=seven_continents[i]
    print(continent)
    lakes_niwBR=os.path.join(r'J:\lakemapping\postprocess\v10_240710\2_polygon_iw_River.gdb','lakes_'+continent+'_niwBR')
    lakes_niwBR_lyr='lakes_'+continent+'_niwBR'
    lakes_niwBR_afm_lyr='lakes_'+continent+'_niwBR_afm'
    lakes_niwBR_afm=os.path.join(after_mask_gdb,'lakes_'+continent+'_niwBR_afterMask')
    arcpy.MakeFeatureLayer_management(lakes_niwBR,lakes_niwBR_lyr)
    selectByAttribute(lakes_niwBR_lyr,'NEW_SELECTION','"operate"<> 0 or operate IS NULL')
    arcpy.conversion.ExportFeatures(lakes_niwBR_lyr,lakes_niwBR_afm)
    arcpy.MakeFeatureLayer_management(lakes_niwBR_afm,lakes_niwBR_afm_lyr)
    out_table=os.path.join(stastics_gdb,f'{stastic_v}_{continent}_niwBR_afm_stastics')
    input_table_afm_list.append(out_table)
    arcpy.analysis.Statistics(lakes_niwBR_afm_lyr, out_table, [['area',"sum"]], ['iwBL']) 
print(input_table_afm_list)

In [ ]:
file_name=f'{stastic_v}_sum_afm_classify_by_Continents_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(input_table_afm_list, Output_Excel_File)

## 2，将mask后结果与big_lakes合并，得到的数据存在4_mwBL_gdb

In [ ]:
merge_list=[lakes_iwBR_afm_lyr]
selectByAttribute(lakes_iwBR_afm_lyr,'NEW_SELECTION',"iwBL = 1")
for continent in seven_continents:
    lakes_niwBR_afm_lyr='lakes_'+continent+'_niwBR_afm'
    lakes_niwBR_afm=os.path.join(after_mask_gdb,'lakes_'+continent+'_niwBR_afterMask')
    arcpy.MakeFeatureLayer_management(lakes_niwBR_afm,lakes_niwBR_afm_lyr)
    merge_list.append(lakes_niwBR_afm_lyr)
    selectByAttribute(lakes_niwBR_afm_lyr,'NEW_SELECTION',"iwBL = 1")
#把iwBL=1的湖泊与big lake文件合并，并融合
merge_list.append(big_lakes_lyr)
print(merge_list)

start=time.time()
arcpy.Merge_management(merge_list,a_iwBL_merge_with_BL_raw)
arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL_raw, a_iwBL_merge_with_BL_raw_lyr)#global_lakes_afm_merged_with_GBL
end=time.time()
print(f'time:{end-start}s')

In [ ]:
start=time.time()
print('start Dissolve')
arcpy.Dissolve_management(a_iwBL_merge_with_BL_raw_lyr, a_iwBL_merge_with_BL,"", "", "SINGLE_PART")#dissolve copylake
arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL, a_iwBL_merge_with_BL_lyr)
arcpy.AddField_management(a_iwBL_merge_with_BL_lyr,"area","Double")
arcpy.CalculateField_management(a_iwBL_merge_with_BL_lyr, "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
end=time.time()
print(f'time:{end-start}s')

In [ ]:
arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL, a_iwBL_merge_with_BL_lyr)
# arcpy.AddField_management(a_iwBL_merge_with_BL_lyr, 'iwBL', "short")
# calculateField(a_iwBL_merge_with_BL_lyr, "iwBL",1)
# arcpy.AddField_management(a_iwBL_merge_with_BL_lyr, 'Continent', "Text")
# cal_continent(a_iwBL_merge_with_BL_lyr)
# arcpy.AddField_management(a_iwBL_merge_with_BL_lyr, 'flag', "short")

In [ ]:
# arcpy.management.Delete(a_iwBL_merge_with_BL_raw)
selectByLocation(a_iwBL_merge_with_BL_lyr, 'INTERSECT', GLAKES_natural_lake)
calculateField(a_iwBL_merge_with_BL_lyr, "flag",1)
selectByLocation(a_iwBL_merge_with_BL_lyr, 'INTERSECT', GLAKES_Res)
calculateField(a_iwBL_merge_with_BL_lyr, "flag",2)

In [ ]:
input_table_b_list=[]
for i in range(0,6):
    continent=seven_continents[i]
    selectByAttribute(a_iwBL_merge_with_BL_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
    
    selectByAttribute(lakes_iwBR_afm_lyr,'NEW_SELECTION',f"iwBL = 0 AND Continent = '{continent}'")
    lakes_niwBR_afm=os.path.join(after_mask_gdb,'lakes_'+continent+'_niwBR_afterMask')
    lakes_niwBR_afm_lyr='lakes_'+continent+'_niwBR_afm'
    arcpy.MakeFeatureLayer_management(lakes_niwBR_afm,lakes_niwBR_afm_lyr)
    
    selectByAttribute(lakes_niwBR_afm_lyr,'NEW_SELECTION',f"iwBL = 0")
    b_lake_afm_merge_with_BL=os.path.join(mwBL_gdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
    b_lake_afm_merge_with_BL_lyr=f'b{i+1}_{continent}_lake_afm_merge_with_BL'
    
    field_list=['area','operate','iwBL','flag']
    fieldmappings = arcpy.FieldMappings()
    fieldmappings_all = arcpy.FieldMappings()
    fieldmappings_all.addTable(lakes_niwBR_afm_lyr)
    for field_i in field_list:
        field_idx = fieldmappings_all.findFieldMapIndex(field_i)
        fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
        
    arcpy.Merge_management([a_iwBL_merge_with_BL_lyr,lakes_niwBR_afm_lyr,lakes_iwBR_afm_lyr],b_lake_afm_merge_with_BL,field_mappings=fieldmappings)
    arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
    #选出和big lake合并的湖泊及原先flag=3、4的湖泊，赋值2
    selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','flag > 2 OR flag = 0 OR flag IS NULL')
    selectByLocation(b_lake_afm_merge_with_BL_lyr, 'INTERSECT', GLAKES_gte1_Rser_lyr,'SUBSET_SELECTION')
    calculateField(b_lake_afm_merge_with_BL_lyr, "flag",2.5)
    selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','')
    
    out_table=os.path.join(stastics_gdb,f'{stastic_v}_{continent}_lakes_merge_with_BL_stastics')
    input_table_b_list.append(out_table)
    arcpy.analysis.Statistics(b_lake_afm_merge_with_BL_lyr, out_table, [['area',"sum"]], ['flag']) 

file_name=f'{stastic_v}_sum_lakes_afm_mwBL_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(input_table_b_list, Output_Excel_File)

## 3，将b结果与big_GLAKES合并，得到的数据存在mwBG.gdb

### 3.1 合并

In [ ]:
merge_list_BG=[]
start=time.time()
for i in range(0,6):
    continent=seven_continents[i]
    b_lake_afm_merge_with_BL=os.path.join(mwBL_gdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
    b_lake_afm_merge_with_BL_lyr=f'b{i+1}_{continent}_lake_afm_merge_with_BL'
    arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
    selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION',"flag > 0 and flag < 3")
    merge_list_BG.append(b_lake_afm_merge_with_BL_lyr)
arcpy.Merge_management(merge_list_BG,b_iwBG)
arcpy.MakeFeatureLayer_management(b_iwBG, b_iwBG_lyr)
arcpy.management.FeatureToPoint(b_iwBG_lyr,b_iwBG_point,'INSIDE')
arcpy.MakeFeatureLayer_management(b_iwBG_point, b_iwBG_point_lyr)
arcpy.Merge_management([b_iwBG_lyr,GLAKES_gte1_Rser_lyr],b_iwBG_merge_with_BG_raw)#GLAKES_gte1_lyr
end=time.time()
print(f'time:{end-start}s')

In [ ]:
start=time.time()
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG_raw, b_iwBG_merge_with_BG_raw_lyr)
print('start Dissolve')
arcpy.Dissolve_management(b_iwBG_merge_with_BG_raw_lyr, b_iwBG_merge_with_BG,"", "", "SINGLE_PART")# dissolve copylake
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG, b_iwBG_merge_with_BG_lyr)
end=time.time()
print(f'time:{end-start}s')

In [ ]:
arcpy.management.Delete(b_iwBG_merge_with_BG_raw)
# arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG, b_iwBG_merge_with_BG_lyr)

### 3.2 匹配湖泊对应的GLAKES Lake_id

In [ ]:
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(GLAKES_gte1_Rser_point_lyr)

#包含的面积最大的GLAKES的Lake_id
field_idx = fieldmappings_all.findFieldMapIndex('Lake_id')
fieldmappings = arcpy.FieldMappings()
fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
Lake_id_idx = fieldmappings.findFieldMapIndex("Lake_id")
Lake_id_fieldmap = fieldmappings.getFieldMap(Lake_id_idx)
Lake_id_fieldmap.mergeRule = "min"
fieldmappings.replaceFieldMap(Lake_id_idx, Lake_id_fieldmap)

print('Conduct spatial join')
arcpy.SpatialJoin_analysis(b_iwBG_merge_with_BG_lyr, GLAKES_gte1_Rser_point_lyr, b_iwBG_merge_with_BG_SJ, "JOIN_ONE_TO_ONE",
                           "KEEP_ALL", fieldmappings, "CONTAINS")
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG_SJ,b_iwBG_merge_with_BG_SJ_lyr)

In [ ]:
##解决GLAKES 八连通湖泊 不一致的问题
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(GLAKES_gte1_Rser_lyr)

field_idx = fieldmappings_all.findFieldMapIndex('Lake_id')
fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
Lake_id_idx = fieldmappings.findFieldMapIndex("Lake_id")
Lake_id_fieldmap = fieldmappings.getFieldMap(Lake_id_idx)
Lake_id_fieldmap.mergeRule = "min"
fieldmappings.replaceFieldMap(Lake_id_idx, Lake_id_fieldmap)

selectByAttribute(b_iwBG_merge_with_BG_SJ_lyr,"NEW_SELECTION","Lake_id IS NULL")
print('Conduct spatial join')
arcpy.SpatialJoin_analysis(b_iwBG_merge_with_BG_SJ_lyr, GLAKES_gte1_Rser_lyr, b_iwBG_merge_with_BG_SJ_SJ, "JOIN_ONE_TO_ONE",
                           "KEEP_ALL", fieldmappings, "LARGEST_OVERLAP")

In [ ]:
arcpy.management.AddJoin(b_iwBG_merge_with_BG_SJ_lyr,'OBJECTID','b_iwBG_merge_with_BG_SJ_SJ','TARGET_FID')
arcpy.CalculateField_management(b_iwBG_merge_with_BG_SJ_lyr,'b_iwBG_merge_with_BG_SJ.Lake_id','!b_iwBG_merge_with_BG_SJ_SJ.Lake_id!')
arcpy.management.RemoveJoin(b_iwBG_merge_with_BG_SJ_lyr)

In [ ]:
arcpy.Dissolve_management(b_iwBG_merge_with_BG_SJ_lyr, b_iwBG_merge_with_BG_SJ_dissolve,"Lake_id")
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG_SJ_dissolve, b_iwBG_merge_with_BG_SJ_dissolve_lyr)
arcpy.AddField_management(b_iwBG_merge_with_BG_SJ_dissolve_lyr,"lake_area","Double")
arcpy.CalculateField_management(b_iwBG_merge_with_BG_SJ_dissolve_lyr, "lake_area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")

### 3.3 统计包含的GLAKES 湖泊面积

In [ ]:
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(b_iwBG_merge_with_BG_SJ_dissolve_lyr)
field_idx = fieldmappings_all.findFieldMapIndex('Lake_id')
fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
fieldmappings_all.addTable(GLAKES_gte1_Rser_point_lyr)
field_list=['lake_area','Area_bound']
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
Area_bound_idx = fieldmappings.findFieldMapIndex("Area_bound")
Area_bound_fieldmap = fieldmappings.getFieldMap(Area_bound_idx)
Area_bound_fieldmap.mergeRule = "sum"
fieldmappings.replaceFieldMap(Area_bound_idx, Area_bound_fieldmap)
arcpy.SpatialJoin_analysis(b_iwBG_merge_with_BG_SJ_dissolve_lyr, GLAKES_gte1_Rser_point_lyr, b_iwBG_merge_with_BG_SJ_GLAKES_SJ, "JOIN_ONE_TO_ONE",
                           "KEEP_ALL", fieldmappings, "CONTAINS")

In [ ]:
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG_SJ_GLAKES_SJ,b_iwBG_merge_with_BG_SJ_GLAKES_SJ_lyr)
arcpy.AddField_management(b_iwBG_merge_with_BG_SJ_GLAKES_SJ_lyr, 'GLAKES_AR', "DOUBLE")#新建字段为面积比例
arcpy.CalculateField_management(b_iwBG_merge_with_BG_SJ_GLAKES_SJ_lyr, 'GLAKES_AR', '!Area_bound!/!lake_area!', "PYTHON3")

### 3.3 统计包含的merge前的湖泊面积

In [ ]:
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(b_iwBG_merge_with_BG_SJ_GLAKES_SJ_lyr)
fieldmappings_all.addTable(b_iwBG_point_lyr)
field_list=['lake_area','Lake_id','GLAKES_AR','area']
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
area_idx = fieldmappings.findFieldMapIndex("area")
area_fieldmap = fieldmappings.getFieldMap(area_idx)
area_fieldmap.mergeRule = "sum"
fieldmappings.replaceFieldMap(area_idx, area_fieldmap)
arcpy.SpatialJoin_analysis(b_iwBG_merge_with_BG_SJ_GLAKES_SJ_lyr, b_iwBG_point_lyr, b_iwBG_merge_with_BG_final, "JOIN_ONE_TO_ONE",
                           "KEEP_ALL", fieldmappings, "CONTAINS")

In [ ]:
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG_final,b_iwBG_merge_with_BG_final_lyr)
# arcpy.AddField_management(b_iwBG_merge_with_BG_final_lyr, 'merge_AR', "DOUBLE")#新建字段为面积比例
# arcpy.CalculateField_management(b_iwBG_merge_with_BG_final_lyr, 'merge_AR', '!area!/!lake_area!', "PYTHON3")

In [ ]:
arcpy.DeleteField_management(b_iwBG_merge_with_BG_final_lyr, 'area')
arcpy.AddField_management(b_iwBG_merge_with_BG_final_lyr, 'flag', "DOUBLE")
selectByAttribute(b_iwBG_merge_with_BG_final_lyr,'NEW_SELECTION',"merge_AR IS NOT NULL")
arcpy.CalculateField_management(b_iwBG_merge_with_BG_final_lyr, 'flag',2.5)
selectByAttribute(b_iwBG_merge_with_BG_final_lyr,'NEW_SELECTION',"merge_AR IS NULL")
selectByLocation(b_iwBG_merge_with_BG_final_lyr, 'INTERSECT', 'GLAKES_Res','SUBSET_SELECTION')
arcpy.CalculateField_management(b_iwBG_merge_with_BG_final_lyr, 'flag',2)
selectByAttribute(b_iwBG_merge_with_BG_final_lyr,'NEW_SELECTION',"merge_AR IS NULL")
arcpy.CalculateField_management(b_iwBG_merge_with_BG_final_lyr, 'flag',1)
cal_continent(b_iwBG_merge_with_BG_final_lyr)

### 3.4 将不与GLAKES相交的湖泊与前面生成的结果合并，得到c

In [ ]:
# input_table_mwBG_list=[]
for i in range(5,6):#[1,3,4,5]:
    continent=seven_continents[i]
    print(continent)
    b_lake_afm_merge_with_BL_lyr=f'b{i+1}_{continent}_lake_afm_merge_with_BL'
    b_lake_afm_merge_with_BL=os.path.join(mwBL_gdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
    arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL,b_lake_afm_merge_with_BL_lyr)
#     arcpy.AddField_management(b_lake_afm_merge_with_BL_lyr, 'lake_area', "DOUBLE")
#     arcpy.CalculateField_management(b_lake_afm_merge_with_BL_lyr, 'lake_area','!area!')
    
    selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION',"flag IS NULL or flag = 0 OR flag>=3")
    selectByAttribute(b_iwBG_merge_with_BG_final_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
    
    c_lake_afm_mw_BL_BG=os.path.join(mwBG_gdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG')
    c_lake_afm_mw_BL_BG_lyr=f'c{i+1}_{continent}_lake_afm_mw_BL_BG'
    
    field_list=['lake_area','Lake_id','GLAKES_AR','merge_AR','flag','operate']
    fieldmappings = arcpy.FieldMappings()
    fieldmappings_all = arcpy.FieldMappings()
    fieldmappings_all.addTable(b_lake_afm_merge_with_BL_lyr)#未mask
    fieldmappings_all.addTable(b_iwBG_merge_with_BG_final_lyr)
    for field_i in field_list:
        field_idx = fieldmappings_all.findFieldMapIndex(field_i)
        fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
    
    arcpy.Merge_management([b_lake_afm_merge_with_BL_lyr,b_iwBG_merge_with_BG_final_lyr],c_lake_afm_mw_BL_BG,field_mappings=fieldmappings)
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_BL_BG,c_lake_afm_mw_BL_BG_lyr)
    selectByAttribute(c_lake_afm_mw_BL_BG_lyr,'NEW_SELECTION',"")
    out_table=os.path.join(stastics_gdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG_stastics_{version}')
    input_table_mwBG_list.append(out_table)
    arcpy.analysis.Statistics(c_lake_afm_mw_BL_BG_lyr, out_table, [['lake_area',"sum"]], ['flag']) 
# file_name=f'sum_lakes_afm_mwBL_BG_stastics_{version}.xls'
# Output_Excel_File=os.path.join(excel_dir,file_name)
# arcpy.conversion.TableToExcel(input_table_mwBG_list, Output_Excel_File)    

In [ ]:
arcpy.analysis.Statistics(c_lake_afm_mw_BL_BG_lyr, out_table, [['lake_area',"sum"]], ['flag']) 

In [ ]:
# input_table_mwBL_list=[]
# input_table_mwBL_BG_list=[]
# for i in range(0,6):
#     b_lyr=b_lyr_list[i]
#     continent=seven_continents[i]
#     out_table_1=os.path.join(stastics_gdb,f'b{i+1}_{continent}_afm_mwBL_by_Continent_stastics')
#     input_table_mwBL_list.append(out_table_1)
#     c_lake_afm_mw_BL_BG=os.path.join( Cgdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG')
#     c_lake_afm_mw_BL_BG_lyr=f'c{i+1}_{continent}_lake_afm_mw_BL_BG'
# #     arcpy.MakeFeatureLayer_management(c_lake_afm_mw_BL_BG,c_lake_afm_mw_BL_BG_lyr)
#     out_table=os.path.join(stastics_gdb,f'c{i+1}_{continent}_afm_mwBL_BG_by_Continent_stastics')
#     input_table_mwBL_BG_list.append(out_table)
# #     arcpy.analysis.Statistics(c_lake_afm_mw_BL_BG_lyr, out_table, [['area',"sum"]]) 
#     out_table_2=os.path.join(stastics_gdb,f'c{i+1}_{continent}_afm_mwBL_BG_by_Continent_stastics')
#     input_table_mwBL_BG_list.append(out_table_2)
    
# Output_Excel_File=r'J:\lakemapping\postprocess\v10_240710\sum_afm_mwBL_classify_by_Continents_stastics.xls'
# Output_Excel_File_2=r'J:\lakemapping\postprocess\v10_240710\sum_afm_mwBL_BG_classify_by_Continents_stastics.xls'
# print(input_table_mwBL_BG_list)
# arcpy.conversion.TableToExcel(input_table_mwBL_list, Output_Excel_File)
# arcpy.conversion.TableToExcel(input_table_mwBL_BG_list, Output_Excel_File_2)